<a href="https://colab.research.google.com/github/syafiyahh/TaskWeek7/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Syafiyah Hamnah Hasan
NPM: 2106726144
Link: https://huggingface.co/alexgeh196/twitter-sentiment-analysis

In [ ]:
!pip install transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("alexgeh196/twitter-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("alexgeh196/twitter-sentiment-analysis")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    sentiment = torch.argmax(probs).item()
    return "Positive" if sentiment == 1 else "Negative"

text = "I love using deep learning models for sentiment analysis!"
print(predict_sentiment(text))


Positive


In [ ]:
import pandas as pd

# Create a synthetic dataset
data = {
    "sentence": [
        "I love this product, it works amazingly well!",  # Positive
        "This was the worst experience of my life.",      # Negative
        "I would definitely recommend this to others!",   # Positive
        "Absolutely terrible service, never coming back.",# Negative
        "A fantastic outcome, exceeded all my expectations.", # Positive
        "So disappointing, not what I expected at all.",   # Negative
        "An excellent product, very satisfied!",           # Positive
        "Worst investment ever, a total waste of money.",  # Negative
        "Truly a remarkable and reliable product!",        # Positive
        "I regret buying this, nothing works as promised." # Negative
    ],
    "expected_sentiment": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}

# Convert to DataFrame
synthetic_dataset = pd.DataFrame(data)

# Save to CSV
file_path = "/content/synthetic_sentiment_dataset.csv"
synthetic_dataset.to_csv(file_path, index=False)

# Provide download link
from google.colab import files
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Test with a sample sentence
sample_sentence = "This product exceeded all my expectations!"
result = predict_sentiment(sample_sentence)
print(f"Sentiment of the sample sentence: {result}")

Sentiment of the sample sentence: Positive


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the dataset
data = pd.read_csv("/content/synthetic_sentiment_dataset.csv")

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("alexgeh196/twitter-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("alexgeh196/twitter-sentiment-analysis")

# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    sentiment = torch.argmax(probs).item()
    return sentiment  # 1 = Positive, 0 = Negative

# Apply the model to each sentence and compare with expected sentiment
correct_predictions = 0

for index, row in data.iterrows():
    predicted = predict_sentiment(row['sentence'])
    actual = row['expected_sentiment']
    if predicted == actual:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / len(data) * 100
print(f"Model Accuracy on Synthetic Dataset: {accuracy:.2f}%")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model Accuracy on Synthetic Dataset: 100.00%


In [ ]:
!pip install torch


In [ ]:
import torch
import torch.nn as nn
import math

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size must be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Scaled Dot-Product Attention
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


In [ ]:
embed_size = 256  # Example embedding size
heads = 8  # Number of attention heads
dropout = 0.1  # Dropout rate
forward_expansion = 4  # Feed-forward network expansion factor
mask = None  # Optional mask for attention

# Instantiate the transformer block
transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)

# Dummy input
x = torch.rand(32, 10, embed_size)  # (batch_size, sequence_length, embed_size)

# Pass through transformer block
out = transformer_block(x, x, x, mask)
print(out.shape)  # Expected output: (32, 10, embed_size)


torch.Size([32, 10, 256])


In [ ]:
# Test with a sample sentence
sample_sentence = "This product exceeded all my expectations!"
result = predict_sentiment(sample_sentence)
print(f"Sentiment of the sample sentence: {result}")

Sentiment of the sample sentence: 1


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.metrics import accuracy_score

# Synthetic Dataset
data = {
    "sentence": [
        "I love this product, it works amazingly well!",  # Positive
        "This was the worst experience of my life.",      # Negative
        "I would definitely recommend this to others!",   # Positive
        "Absolutely terrible service, never coming back.",# Negative
        "A fantastic outcome, exceeded all my expectations.", # Positive
        "So disappointing, not what I expected at all.",   # Negative
        "An excellent product, very satisfied!",           # Positive
        "Worst investment ever, a total waste of money.",  # Negative
        "Truly a remarkable and reliable product!",        # Positive
        "I regret buying this, nothing works as promised." # Negative
    ],
    "expected_sentiment": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}
df = pd.DataFrame(data)

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("alexgeh196/twitter-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("alexgeh196/twitter-sentiment-analysis")

# Predict function
def predict_sentiment(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    prediction = torch.argmax(probs).item()
    return prediction  # 1 = Positive, 0 = Negative

# Apply predictions to the dataset
df["predicted_sentiment"] = df["sentence"].apply(predict_sentiment)

# Calculate accuracy
accuracy = accuracy_score(df["expected_sentiment"], df["predicted_sentiment"])
print(f"Model Accuracy on Synthetic Dataset: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model Accuracy on Synthetic Dataset: 1.00
